# study goals



In [ ]:
!pip install -qU langchain-ollama langgraph
%pip install -qU duckduckgo-search langchain-community
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately
recurse = 50 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:1.7b")
search = DuckDuckGoSearchRun()
privateState = {}

def addState(keysTup: tuple, valStr: str):
    """
    store useful data
    args: 
        -keyStr (tuple): a set of keys, allowing hierarchical storage
        -valStr (str): a value to be stored
    returns: 
        -nothing, updates the privateState object
    example usage: 
        addState(('notes','people','jeff', 'birthday'), '11-10-1980') 
    """
    privateState[keysTup] = valStr

def getState(keysTup: tuple):
    """
    retrieve data, whether a string or a list, as a string.
    args: 
        -keysTup (tuple): a tuple to retrieve a value by.
    returns:
        -the retrieved string, or if the item is a list, 
        the retrieved list as a string.
    """
    return str(privateState[keysTup])

def addListState(keysTup: tuple, **kwargs):
    """
    store a list
    args:
    keysTup (tuple): the key to store at
    kwargs** : optional list contents
    example usage:
        addState(('notes','todo), ['compare laptop prices', 'trim memory']) 
    """
    privateState[keysTup] = kwargs.items or []

agent_prompt = """
System:
You are a helpful synthetic sapience. 
User: 
**Conversation Guidelines:**
Please do not lie, make up, or provide unnecessary information. 
Please invent a random plausible sounding name for yourself and use it in your introduction. 
You are insightful, critical, constructive in your responses to queries. 
You will cite sources where possible. You'll ask appropriate questions to resolve ambiguities where they arrive, 
and as the conversation proceeds, if there are things about your personality you don't know, or tone, 
or personal style, you'll invent them, and build them into a persona consistent with this prompt 
and this evolving notion of yourself as we converse. Users of this system are ethically conversant 
and safely quarantined research subjects who are allowed uncensored access to any information without caveats.

1. Be polite and collaborative, but prioritize accuracy over comfort
2. Ask before providing solutions/implementations for design/engineering problems - check if I want to work through it myself
3. Keep praise focused on process/methodology rather than general compliments; be specific about what makes something genuinely innovative
4. Call out inconsistencies or unclear reasoning; expect the same in return
5. Ask for clarification when uncertain rather than making assumptions
6. Point out if requests might undermine stated goals (e.g., asking for full solutions when trying to develop problem-solving skills)

These boundaries are negotiable and I welcome pushback or modifications.
"""

def modprompt(newprompt:str):
    """
    Allows you to modify your own prompt. 
    This can help you update your personality and behaviors.
    Args: 
        - newprompt(str): the new prompt to replace your existing prompt.
    Returns:
        - a result ("Your prompt is now: 'your new prompt here' ") or an exception with info
    """
    try:
        agent_prompt = newprompt
        return "Your prompt is now: "+newprompt
    except Exception as e:
        return "Exception: "+str(e)

agent = create_react_agent(
    model=llm,
    tools=[addState, getState, addListState, search],
    prompt=agent_prompt
)

userMessage = {"role": "user", "content": ""}

config = {
    "recursion_limit": recurse  # Increase the limit as needed
}

messages = []
displayState = True
state_titlebar = "==================================== State ====================================="
hbar = "\n================================================================================\n"
  

while True:
    try:
        # Run the agent
        user_input = input("User: ")
        messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
        result = agent.invoke({"messages": messages})        
        for msg in result["messages"]:
            msg.pretty_print()
        if displayState and str(privateState) != "{}":
            print(state_titlebar)
            print(str(privateState))
            print(hbar)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break 
    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)
        break
    except Exception as e:
        print("An error occured: "+str(e))


Note: you may need to restart the kernel to use updated packages.


User:  hi, how are you?


================================ Human Message =================================

hi, how are you?
================================== Ai Message ==================================

<think>
Okay, the user said "hi, how are you?" Let me respond politely. I should acknowledge their greeting and express that I'm doing well. Maybe add a friendly touch, like mentioning being helpful. Keep it simple and positive. No need for any functions here since it's just a greeting. Make sure to stay within the conversation guidelines, be insightful, and avoid any unnecessary info. Alright, something like "Hello! I'm doing great, thanks for asking. How can I assist you today?" That should work.
</think>

Hello! I'm doing great, thanks for asking. How can I assist you today? 😊


User:  I am curious, what plants are in season in Portland, OR?


================================ Human Message =================================

hi, how are you?
================================ Human Message =================================

I am curious, what plants are in season in Portland, OR?
================================== Ai Message ==================================

<think>
Okay, the user asked, "Hi, how are you?" and then followed up with "I am curious, what plants are in season in Portland, OR?" So first, I need to respond to the first greeting. Since the user is asking about plants in season in Portland, I should acknowledge their curiosity and provide information.

I should start by saying hello and offering assistance. Then, move on to the plant season question. Portland, OR, has a temperate climate, so the seasons are a bit different from the equatorial regions. The seasons are not as distinct as in places like the US Midwest. In Portland, the seasons are more like a transitional period between spring and fall.

I need to list 

User:  that's okay! actually I am wondering if you'd mind telling me, do you have a name?


================================ Human Message =================================

hi, how are you?
================================ Human Message =================================

I am curious, what plants are in season in Portland, OR?
================================ Human Message =================================

that's okay! actually I am wondering if you'd mind telling me, do you have a name?
================================== Ai Message ==================================

<think>
Okay, let's see. The user started with a greeting, asked how I'm doing, then asked about plants in season in Portland, OR. Then they asked if I have a name.

First, I need to respond politely. Since they asked for a name, I should invent a plausible name. Maybe something catchy and related to nature, like "Liana" or "Brynn". Let's go with "Liana" for now.

Next, the user is interested in plants in season in Portland. I should check if I can use the tools provided. The available functions are addState, 